## Generating BiomedCLIP Embeddings

In [4]:
import os 
import nibabel as nib
import numpy as np
import SimpleITK as sitk
import json
import pandas as pd
from tqdm import tqdm
from PIL import Image
import pickle
import sys
import open_clip
import torch
from torch.utils.data import Dataset

# Load Model
model, preprocess_train, preprocess_val = open_clip.create_model_and_transforms('hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224')
tokenizer = open_clip.get_tokenizer('hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224')

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)
model.to(device)

# CT scan dataset
class LoadDataset(Dataset):
    def __init__(self, path, preprocess_fn):
        self.path = path
        self.preprocess_fn = preprocess_fn
        self.volume = self._load_volume(self.path)        
        self.slices = [self.volume[i] for i in range(self.volume.shape[0])]

    def _load_volume(self, path):
        vol = sitk.ReadImage(path)
        vol = sitk.GetArrayFromImage(vol)
        return vol

    def __len__(self):
        return len(self.slices)
    
    def _preprocess_image(self, im):
        return self.preprocess_fn(Image.fromarray(im))

    def __getitem__(self, index):
        return {
            "image": self._preprocess_image(self.slices[index])
            }

## Generate Embeddings with BiomedCLIP

In [6]:
# Input Nifti Folder
dir_path = os.path.join(r'C:\Users\ADMIN\Desktop\Workspace\Code\data\msd_segmentation\Task07_Pancreas\imagesTr')

## Output embedding directory
output_dir = r'C:\Users\ADMIN\Desktop\Workspace\Code\data\embedding'
os.makedirs(output_dir, exist_ok=True)

In [1]:
# CT scan dataset
class LoadDataset(Dataset):
    def __init__(self, path, preprocess_fn):
        self.path = path
        self.preprocess_fn = preprocess_fn
        self.volume = self._load_volume(self.path)        
        self.slices = [self.volume[i] for i in range(self.volume.shape[0])]

    def _load_volume(self, path):
        vol = sitk.ReadImage(path)
        vol = sitk.GetArrayFromImage(vol)
        return vol

    def __len__(self):
        return len(self.slices)
    
    def _preprocess_image(self, im):
        return self.preprocess_fn(Image.fromarray(im))

    def __getitem__(self, index):
        return {
            "image": self._preprocess_image(self.slices[index])
            }
    
train_data = []

for img in os.listdir(dir_path):
    if '.amlignore' in img:
        pass
    elif '.DS_Store' in img:
        pass
    elif '._.DS_Store' in img:
        pass
    elif '._' in img:
        pass
    else:
        img_path = os.path.join(dir_path, img)
        train_data.append(img_path)

for scan_data in tqdm(train_data):    
    print(scan_data)
    image_path = scan_data
    image_name = image_path.split('/')[-1].split('.')[0]

    # Load CT scan
    volume_path = scan_data

    # Create dataloader for the CT scan
    dataset = LoadDataset(volume_path, preprocess_val)
    dataloader = torch.utils.data.DataLoader(
        dataset,
        pin_memory=False,
        shuffle=False,
        num_workers=2,
        drop_last=False,
        batch_size=32
    )

    # Generate embeddings (not normalized)
    embds = []
    for batch in dataloader:
        images = batch["image"].to(device) 
        embd = model(images)[0].gpu().detach().numpy()
        embds.append(embd)
        
    embds = np.concatenate(embds, axis=0)
    with open(os.path.join(output_dir, f"{image_name}.pkl"), "wb") as f:
        pickle.dump(embds, f)

c:\Users\ADMIN\Desktop\Workspace\Code\.venv\Lib\site-packages\open_clip\factory.py:129: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path

C:\Users\ADMIN\Desktop\Workspace\Code\data\msd_segmentation\Task07_Pancreas\imagesTr\pancreas_001.nii.gz
